In [ ]:
# Tweepy module written by Josh Roselin, documentation at https://github.com/tweepy/tweepy
# MySQLdb module written by Andy Dustman, documentation at http://mysql-python.sourceforge.net/MySQLdb.html
# GeoSearch crawler written by Chris Cantey, MS GIS/Cartography, University of Wisconsin, https://geo-odyssey.com
# MwSQLdb schema written with great assistance from Steve Hemmy, UW-Madison DoIT


from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import time
import MySQLdb
import random

# Go to http://dev.twitter.com and create an app. 
# The consumer key and secret as well as the access_token and secret will be generated for you after you register with Twitter Developers
consumer_key="rw9f6wbVoLbfU5YvAEEWUDgvh"
consumer_secret="2kmsGR9K7M3KOle0ZQAdDI0fBNFoq9JyuzMo0l80e4hKW5jYcS"

access_token="303646546-udSrlRXP325SZEGEsrrPgtxHLHen5bEVZYjwkyml"
access_token_secret="QXtDD7bhBklrI5qj5A8LYPT0d0SujYqjEUtorM2bU8yBH"

# Create your MySQL schema and connect to database, ex: mysql> SET PASSWORD FOR 'root'@'localhost' = PASSWORD('newpwd');
db=MySQLdb.connect(host='ubuntu', user='root', passwd='Pussinboot5', db='twitterfeed') 
db.set_character_set('utf8')

Coords = dict()
Place = dict()
PlaceCoords = dict()
XY = []
curr=db.cursor()

class StdOutListener(StreamListener):
                """ A listener handles tweets that are the received from the stream. 
                This is a basic listener that inserts tweets into MySQLdb.
                """
                def on_status(self, status):
                                #print "Tweet Text: ",status.text
                                text = status.text
                                #print "Time Stamp: ",status.created_at
                                try:
                                    Coords.update(status.coordinates)
                                    XY = (Coords.get('coordinates'))  #Place the coordinates values into a list 'XY'
                                    #print "X: ", XY[0]
                                    #print "Y: ", XY[1]
                                except:
                                    #Often times users opt into 'place' which is neighborhood size polygon
                                    #Calculate center of polygon
                                    Place.update(status.place)
                                    PlaceCoords.update(Place['bounding_box'])
                                    Box = PlaceCoords['coordinates'][0]
                                    XY = [(Box[0][0] + Box[2][0])/2, (Box[0][1] + Box[2][1])/2]
                                    #print "X: ", XY[0]
                                    #print "Y: ", XY[1] 
                                    pass
                                # Comment out next 4 lines to avoid MySQLdb to simply read stream at console
                                # Name of table inside MySQL database 'twitterfeed' is twitter_2
                                curr.execute("""INSERT INTO twitter_2 (UserID, Date, X, Y, Text) VALUES
                                    (%s, %s, %s, %s, %s);""",
                                    (status.id_str,status.created_at,XY[0],XY[1],text))
                                db.commit()
                      

                def main():
                    l = StdOutListener()    
                    auth = OAuthHandler(consumer_key, consumer_secret)
                    auth.set_access_token(access_token, access_token_secret)
                    stream = Stream(auth, l, timeout=30.0)
                    #Only records 'locations' OR 'tracks', NOT 'tracks (keywords) with locations'
                    while True:
                        try:
                            # Call tweepy's userstream method 
                            # Use either locations or track, not both
                            # Specify your search term. There are two options but you can only choose one.
                            # So, the other one must be hashed(#) out before running it.

                            # 1. Compile tweets with specific words including # tags using the 'track' feature
                            #e.g. stream.filter(track=['trump'])## This will feed the stream all mentions of 'keyword' 
                            stream.filter(track=["car"])
                            
                            # 2. Compile tweets within a specific location or geographical region
                            stream.filter(locations=[-125,25,-65,48], async=False)##These coordinates are approximate bounding box around USA
                            
                            break
                        except Exception, e:
                             # Abnormal exit: Reconnect
                             nsecs=random.randint(60,63)
                             time.sleep(nsecs)            

                if __name__ == '__main__':
                    main()